In [1]:
import os
import sys
import pickle
import psutil
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split

# Define data path
DATA_PATH = "data/"
CHECKPOINT_PATH = "models/"

In [2]:
pids = pickle.load(open(os.path.join(DATA_PATH,'pids.pkl'), 'rb'))
vids = pickle.load(open(os.path.join(DATA_PATH,'vids.pkl'), 'rb'))
targs = pickle.load(open(os.path.join(DATA_PATH,'targets.pkl'), 'rb'))
prob_targs = pickle.load(open(os.path.join(DATA_PATH, 'prob_targets_allvisits.pkl'), 'rb'))
seqs = pickle.load(open(os.path.join(DATA_PATH,'seqs.pkl'), 'rb'))
diags = pickle.load(open(os.path.join(DATA_PATH,'diags.pkl'), 'rb'))
codes = pickle.load(open(os.path.join(DATA_PATH,'icd9.pkl'), 'rb'))
categories = pickle.load(open(os.path.join(DATA_PATH,'categories.pkl'), 'rb'))
sub_categories = pickle.load(open(os.path.join(DATA_PATH,'subcategories.pkl'), 'rb'))
assert len(pids) == len(vids) == len(targs) == len(seqs)

In [3]:
len(prob_targs[0][1])

184

In [4]:
targs[0]

[[143, 146, 145], [143, 146, 145]]

In [60]:
class CustomDataset(Dataset):
    
    def __init__(self, seqs, targets):
        self.x = seqs
        self.y = targets
    
    def __len__(self):
        return(len(self.x))
    
    def __getitem__(self, index):
        return (self.x[index], self.y[index])

In [61]:
dataset = CustomDataset(seqs, prob_targs)

In [62]:
def collate_fn(data):
    """
    Arguments:
        data: a list of samples fetched from `CustomDataset`
        
    Outputs:
        x: a tensor of shape (# patiens, max # visits, max # diagnosis codes) of type torch.long
        x_masks: a tensor of shape (# patiens, max # visits, max # diagnosis codes) of type torch.bool
        y: a tensor of shape (# patiens, max # categories) of type torch.long
        y_masks: a tensor of shape (# patiens, max # categories) of type torch.bool
    """
    sequences, targets = zip(*data)

    num_patients = len(sequences)
    num_visits = [len(patient) for patient in sequences]
    num_codes = [len(visit) for patient in sequences for visit in patient]
    batch_num_categories = [len(visit) for patient in targets for visit in patient]
    max_num_visits = max(num_visits)
    max_num_codes = max(num_codes)
#     max_num_categories = max(batch_num_categories)
    max_num_categories = len(targets[0][0])
    
    x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    x_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
#     y = torch.zeros((num_patients, max_num_categories), dtype=torch.long)
    y = torch.zeros((num_patients, max_num_categories), dtype=torch.float)
    y_masks = torch.zeros((num_patients, max_num_categories), dtype=torch.bool)
    for i_patient, patient in enumerate(sequences):   
        for j_visit, visit in enumerate(patient[:-1]):
            for k_code, code in enumerate(visit):
                x[i_patient, j_visit, k_code] = code
                x_masks[i_patient, j_visit, k_code] = 1

#     for i_patient, patient in enumerate(targets):
# #         import pdb; pdb.set_trace()
#         last_visit = patient[-1]
#         y[i_patient,:len(last_visit)] = torch.LongTensor(last_visit)
#         y_masks[i_patient,:len(last_visit)] = torch.BoolTensor(np.ones(len(last_visit)))
    for i_patient, patient in enumerate(targets):
#         import pdb; pdb.set_trace()
        last_visit = patient[-1]
        y[i_patient] = torch.FloatTensor(last_visit)
        y_masks[i_patient] = torch.BoolTensor(np.ones(max_num_categories))
    
    return x, x_masks, y, y_masks

In [63]:
train_split = int(len(dataset)*0.75)
test_split = int(len(dataset)*0.15)
val_split = int(len(dataset)*0.10)

In [64]:
train_split = int(len(dataset)*0.75)
test_split = int(len(dataset)*0.15)

lengths = [train_split, test_split, len(dataset) - (train_split + test_split)]
train_dataset, test_dataset, val_dataset = random_split(dataset, lengths)

print("Length of train dataset:", len(train_dataset))
print("Length of test dataset:", len(test_dataset))
print("Length of val dataset:", len(val_dataset))

Length of train dataset: 6561
Length of test dataset: 1312
Length of val dataset: 875


In [65]:
def load_data(train_dataset, test_dataset, val_dataset, collate_fn):
    '''
    Arguments:
        train_dataset: train dataset of type `CustomDataset`
        test_dataset: test dataset of type `CustomDataset`
        val_dataset: validation dataset of type `CustomDataset`
        collate_fn: collate function
        
    Outputs:
        train_loader, test_loader, val_loader: train, test and validation dataloaders
    '''
    batch_size = 100
    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size,
                                               collate_fn=collate_fn,
                                               shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset,
                                           batch_size=batch_size,
                                           collate_fn=collate_fn,
                                           shuffle=False)
    val_loader = torch.utils.data.DataLoader(val_dataset,
                                             batch_size=batch_size,
                                             collate_fn=collate_fn,
                                             shuffle=False)
    
    return train_loader, test_loader, val_loader


train_loader, test_loader, val_loader = load_data(train_dataset, test_dataset, val_dataset, collate_fn)

In [66]:
def sum_embeddings_with_mask(x, masks):
    """
    Arguments:
        x: the embeddings of diagnosis sequence of shape (batch_size, # visits, # diagnosis codes,
            embedding_dim)
        masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

    Outputs:
        sum_embeddings: the sum of embeddings of shape (batch_size, # visits, embedding_dim)
    """
    x[~masks] = 0
    return x.sum(2)

In [67]:
def get_last_visit(hidden_states, masks):
    """
    Arguments:
        hidden_states: the hidden states of each visit of shape (batch_size, # visits, embedding_dim)
        masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

    Outputs:
        last_hidden_state: the hidden state for the last true visit of shape (batch_size, embedding_dim)
    """
    idx_vector = masks.any(dim=2).sum(1) - 1
    p_idx = torch.arange(0,len(hidden_states), dtype=torch.int64)
    last_hidden_state = hidden_states[p_idx,idx_vector]
    return last_hidden_state

In [75]:
class BaselineRNN(nn.Module):
    
    def __init__(self, num_codes, num_categories):
        super().__init__()
        """
        Arguments:
            num_codes: total number of diagnosis codes
            num_categories: total number of higher level categories to predict
        """
        self.embedding = nn.Embedding(num_codes, embedding_dim=180)
        self.rnn = nn.GRU(180, hidden_size=128, batch_first=True)
        self.fc = nn.Linear(128, num_categories)
    
    def forward(self, x, masks):
        """
        Arguments:
            x: the diagnosis sequence of shape (batch_size, # visits, # diagnosis codes)
            masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

        Outputs:
            logits: logits of shape (batch_size, # categories)
        """
        x = self.embedding(x)
        x = sum_embeddings_with_mask(x, masks)
        output, _ = self.rnn(x)
        true_h_n = get_last_visit(output, masks)
        logits = self.fc(true_h_n)        
        return logits

# load the model here
baseline_rnn = BaselineRNN(num_codes = len(codes), num_categories=len(sub_categories))
baseline_rnn

BaselineRNN(
  (embedding): Embedding(4903, 180)
  (rnn): GRU(180, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=184, bias=True)
)

In [76]:
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(baseline_rnn.parameters(), lr=0.001)
optimizer = torch.optim.Adadelta(baseline_rnn.parameters(), weight_decay=0.001)

In [2]:
def eval_model(model, test_loader, k=15, n=-1):
    """
    Arguments:
        model: the BaselineRNN model
        test_loader: validation dataloader
        k: value for top k predictions
        n: num of records to evaluate in the batch, value -1 evaulates all records
        
    Outputs:
        precision_k: visit-level precison@k
        accuracy_k: code-level accuracy@k
    """
    y_pred = torch.LongTensor()
    y_true = torch.LongTensor()
    all_precision = []
    all_accuracy = []
    
    model.eval()
    with torch.no_grad():
        for x, masks, y, y_masks in test_loader:
            n_eval = y.shape[0] - 1 if n == -1 else n
            y_hat = model(x, masks)
            y_hat = F.softmax(y_hat, dim=-1)
            #y_multihot = indices_to_multihot(y, y_masks, y_hat)
            nz_rows, nz_cols = torch.nonzero(y, as_tuple=True)
            k_correct = 0
            total_precision = 0
            total_accuracy = 0
            for i in range(n_eval):
                visit_correct = 0
                #y_true = y[i, y_masks[i]]
                y_true = nz_cols[nz_rows == i]
                _, y_pred = torch.topk(y_hat[i], k)
                for v in y_true:
                    if v in y_pred:
                        visit_correct += 1
                visit_precision = visit_correct / min(k, len(y_true))
                visit_accuracy = visit_correct / len(y_true)
                k_correct += visit_correct
                total_precision += visit_precision
                total_accuracy += visit_accuracy
            precision_k = total_precision / n_eval
            accuracy_k = total_accuracy / n_eval
            all_precision.append(precision_k)
            all_accuracy.append(accuracy_k)
            
    total_precision_k = np.mean(all_precision)
    total_accuracy_k = np.mean(all_accuracy)
    return total_precision_k, total_accuracy_k

In [82]:
def train(model, train_loader, val_loader, n_epochs):
    """
    Arguments:
        model: the BaselineRNN model
        train_loader: training dataloder
        val_loader: validation dataloader
        n_epochs: total number of epochs
    """
    max_cpu, max_ram = print_cpu_usage()
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        for x, x_masks, y, y_masks in train_loader:
            y_hat = model(x, x_masks)                  
#             y_mh = indices_to_multihot(y, y_masks, y_hat)
            loss = criterion(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        cpu, ram = print_cpu_usage()
        max_cpu = cpu if cpu > max_cpu else max_cpu
        max_ram = ram if ram > max_ram else max_ram
        print(f'Epoch: {epoch+1} \t Training Loss: {train_loss:.6f}')
        for k in range(5, 31, 5):
            precision_k, accuracy_k = eval_model(model, val_loader, k=k)
            print(f'Epoch: {epoch+1} \t Validation precision@k{k}: {precision_k:.4f}, accuracy@k{k}: {accuracy_k:.4f}')
    final_cpu, final_ram = print_cpu_usage()
    print(f"Max CPU usage: {max_cpu:.3f}\tMax RAM % usage: {max_ram}")

In [83]:
def indices_to_multihot(indices, masks, y_hat):
    multihot = torch.zeros_like(y_hat, dtype=torch.float)
    for idx, row in enumerate(indices):
        y_idx = row[masks[idx]].unique()
        multihot[idx] = F.one_hot(y_idx, y_hat.shape[1]).sum(0).float()
    return multihot

In [84]:
def print_cpu_usage():
    load = psutil.getloadavg()[2]
    cpu_usage = (load/os.cpu_count()) * 100
    ram = psutil.virtual_memory()[2]
    print(f"CPU: {cpu_usage:0.2f}")
    print(f"RAM %: {ram}")
    return cpu_usage, ram

In [85]:
n_epochs = 100
%time train(baseline_rnn, train_loader, val_loader, n_epochs)

CPU: 16.08
RAM %: 64.7
CPU: 16.37
RAM %: 65.8
Epoch: 1 	 Training Loss: 4.088585
Epoch: 1 	 Validation precision@k5: 0.6084, accuracy@k5: 0.3186
Epoch: 1 	 Validation precision@k10: 0.5428, accuracy@k10: 0.4619
Epoch: 1 	 Validation precision@k15: 0.5895, accuracy@k15: 0.5713
Epoch: 1 	 Validation precision@k20: 0.6573, accuracy@k20: 0.6543
Epoch: 1 	 Validation precision@k25: 0.7176, accuracy@k25: 0.7175
Epoch: 1 	 Validation precision@k30: 0.7699, accuracy@k30: 0.7699
CPU: 16.38
RAM %: 66.3
Epoch: 2 	 Training Loss: 3.740911
Epoch: 2 	 Validation precision@k5: 0.6286, accuracy@k5: 0.3336
Epoch: 2 	 Validation precision@k10: 0.5805, accuracy@k10: 0.4960
Epoch: 2 	 Validation precision@k15: 0.6159, accuracy@k15: 0.5973
Epoch: 2 	 Validation precision@k20: 0.6831, accuracy@k20: 0.6800
Epoch: 2 	 Validation precision@k25: 0.7387, accuracy@k25: 0.7386
Epoch: 2 	 Validation precision@k30: 0.7866, accuracy@k30: 0.7866
CPU: 16.62
RAM %: 66.0
Epoch: 3 	 Training Loss: 3.653741
Epoch: 3 	 Vali

CPU: 17.80
RAM %: 66.1
Epoch: 19 	 Training Loss: 3.390705
Epoch: 19 	 Validation precision@k5: 0.6738, accuracy@k5: 0.3627
Epoch: 19 	 Validation precision@k10: 0.6337, accuracy@k10: 0.5437
Epoch: 19 	 Validation precision@k15: 0.6683, accuracy@k15: 0.6485
Epoch: 19 	 Validation precision@k20: 0.7261, accuracy@k20: 0.7230
Epoch: 19 	 Validation precision@k25: 0.7769, accuracy@k25: 0.7768
Epoch: 19 	 Validation precision@k30: 0.8201, accuracy@k30: 0.8201
CPU: 17.81
RAM %: 68.4
Epoch: 20 	 Training Loss: 3.386985
Epoch: 20 	 Validation precision@k5: 0.6749, accuracy@k5: 0.3617
Epoch: 20 	 Validation precision@k10: 0.6353, accuracy@k10: 0.5440
Epoch: 20 	 Validation precision@k15: 0.6731, accuracy@k15: 0.6531
Epoch: 20 	 Validation precision@k20: 0.7272, accuracy@k20: 0.7239
Epoch: 20 	 Validation precision@k25: 0.7780, accuracy@k25: 0.7778
Epoch: 20 	 Validation precision@k30: 0.8230, accuracy@k30: 0.8230
CPU: 18.29
RAM %: 68.3
Epoch: 21 	 Training Loss: 3.386311
Epoch: 21 	 Validation 

Epoch: 36 	 Validation precision@k30: 0.8324, accuracy@k30: 0.8324
CPU: 18.99
RAM %: 66.4
Epoch: 37 	 Training Loss: 3.352098
Epoch: 37 	 Validation precision@k5: 0.7119, accuracy@k5: 0.3840
Epoch: 37 	 Validation precision@k10: 0.6585, accuracy@k10: 0.5641
Epoch: 37 	 Validation precision@k15: 0.6926, accuracy@k15: 0.6721
Epoch: 37 	 Validation precision@k20: 0.7483, accuracy@k20: 0.7450
Epoch: 37 	 Validation precision@k25: 0.7965, accuracy@k25: 0.7964
Epoch: 37 	 Validation precision@k30: 0.8358, accuracy@k30: 0.8358
CPU: 19.31
RAM %: 68.5
Epoch: 38 	 Training Loss: 3.347942
Epoch: 38 	 Validation precision@k5: 0.7142, accuracy@k5: 0.3836
Epoch: 38 	 Validation precision@k10: 0.6611, accuracy@k10: 0.5658
Epoch: 38 	 Validation precision@k15: 0.6947, accuracy@k15: 0.6738
Epoch: 38 	 Validation precision@k20: 0.7488, accuracy@k20: 0.7454
Epoch: 38 	 Validation precision@k25: 0.7956, accuracy@k25: 0.7955
Epoch: 38 	 Validation precision@k30: 0.8357, accuracy@k30: 0.8357
CPU: 19.27
RAM 

Epoch: 54 	 Validation precision@k25: 0.8037, accuracy@k25: 0.8035
Epoch: 54 	 Validation precision@k30: 0.8443, accuracy@k30: 0.8443
CPU: 22.10
RAM %: 65.7
Epoch: 55 	 Training Loss: 3.308905
Epoch: 55 	 Validation precision@k5: 0.7269, accuracy@k5: 0.3890
Epoch: 55 	 Validation precision@k10: 0.6703, accuracy@k10: 0.5740
Epoch: 55 	 Validation precision@k15: 0.7066, accuracy@k15: 0.6853
Epoch: 55 	 Validation precision@k20: 0.7610, accuracy@k20: 0.7576
Epoch: 55 	 Validation precision@k25: 0.8055, accuracy@k25: 0.8053
Epoch: 55 	 Validation precision@k30: 0.8442, accuracy@k30: 0.8442
CPU: 22.09
RAM %: 65.8
Epoch: 56 	 Training Loss: 3.307431
Epoch: 56 	 Validation precision@k5: 0.7233, accuracy@k5: 0.3887
Epoch: 56 	 Validation precision@k10: 0.6740, accuracy@k10: 0.5769
Epoch: 56 	 Validation precision@k15: 0.7048, accuracy@k15: 0.6836
Epoch: 56 	 Validation precision@k20: 0.7590, accuracy@k20: 0.7556
Epoch: 56 	 Validation precision@k25: 0.8081, accuracy@k25: 0.8080
Epoch: 56 	 Val

Epoch: 72 	 Validation precision@k20: 0.7570, accuracy@k20: 0.7537
Epoch: 72 	 Validation precision@k25: 0.8075, accuracy@k25: 0.8073
Epoch: 72 	 Validation precision@k30: 0.8445, accuracy@k30: 0.8445
CPU: 23.21
RAM %: 65.7
Epoch: 73 	 Training Loss: 3.283145
Epoch: 73 	 Validation precision@k5: 0.7158, accuracy@k5: 0.3856
Epoch: 73 	 Validation precision@k10: 0.6726, accuracy@k10: 0.5759
Epoch: 73 	 Validation precision@k15: 0.7107, accuracy@k15: 0.6892
Epoch: 73 	 Validation precision@k20: 0.7607, accuracy@k20: 0.7573
Epoch: 73 	 Validation precision@k25: 0.8111, accuracy@k25: 0.8110
Epoch: 73 	 Validation precision@k30: 0.8465, accuracy@k30: 0.8465
CPU: 23.15
RAM %: 65.7
Epoch: 74 	 Training Loss: 3.280448
Epoch: 74 	 Validation precision@k5: 0.7142, accuracy@k5: 0.3833
Epoch: 74 	 Validation precision@k10: 0.6703, accuracy@k10: 0.5743
Epoch: 74 	 Validation precision@k15: 0.7067, accuracy@k15: 0.6858
Epoch: 74 	 Validation precision@k20: 0.7604, accuracy@k20: 0.7571
Epoch: 74 	 Val

Epoch: 90 	 Validation precision@k15: 0.7081, accuracy@k15: 0.6872
Epoch: 90 	 Validation precision@k20: 0.7578, accuracy@k20: 0.7544
Epoch: 90 	 Validation precision@k25: 0.8061, accuracy@k25: 0.8059
Epoch: 90 	 Validation precision@k30: 0.8443, accuracy@k30: 0.8443
CPU: 23.93
RAM %: 66.1
Epoch: 91 	 Training Loss: 3.265081
Epoch: 91 	 Validation precision@k5: 0.7204, accuracy@k5: 0.3880
Epoch: 91 	 Validation precision@k10: 0.6756, accuracy@k10: 0.5790
Epoch: 91 	 Validation precision@k15: 0.7063, accuracy@k15: 0.6855
Epoch: 91 	 Validation precision@k20: 0.7609, accuracy@k20: 0.7575
Epoch: 91 	 Validation precision@k25: 0.8103, accuracy@k25: 0.8101
Epoch: 91 	 Validation precision@k30: 0.8468, accuracy@k30: 0.8468
CPU: 23.83
RAM %: 66.1
Epoch: 92 	 Training Loss: 3.265816
Epoch: 92 	 Validation precision@k5: 0.7090, accuracy@k5: 0.3807
Epoch: 92 	 Validation precision@k10: 0.6676, accuracy@k10: 0.5718
Epoch: 92 	 Validation precision@k15: 0.7039, accuracy@k15: 0.6828
Epoch: 92 	 Val

In [3]:
for k in range(5, 31, 5):
    precision_k, accuracy_k = eval_model(baseline_rnn, test_loader, k=k)
    print(f'Validation precision@k{k}: {precision_k:.4f}, accuracy@k{k}: {accuracy_k:.4f}')

NameError: name 'baseline_rnn' is not defined

In [ ]:
torch.save(baseline_rnn, os.path.join(CHECKPOINT_PATH, "BaselineRNN_100.pth"))